Testnet here: https://testnet.binancefuture.com/en/futures/BTCUSDT

In [1]:
api_key = "6ce63f3406fd8ebbff01054a66c25fe3c851c45932088c8ca3131a7005188462"
secret_key = "aa3ea32929252467fa5ffeac5818c95beabfb5dba691ef445e7eaa31ea0d15f6"

In [2]:
from lib.TechnicalIndicators import *
from binance.client import Client
from binance.websocket.cm_futures.websocket_client import CMFuturesWebsocketClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

2023-01-07 09:09:31.242410: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 09:09:31.819946: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-07 09:09:31.820004: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-07 09:09:31.979405: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-07 09:09:34.033333: W tensorflow/stream_executor/platform/default

In [1]:
class FuturesTrader():
    def __init__(self, symbol="btcusd", testnet = True):
        ####API CONNECTIONS ####
        self.stream = None
        self.client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = testnet)
        #######################
        self.data = None
        self.symbol = symbol
        self.asset = self.get_asset(self.symbol) #get asset like "USDT"
        #### "BTCUSD" is not valid with official api methods, need to use "BTCUSDT" ####
        symbol = self.symbol + "t" if self.symbol.endswith('usd') else self.symbol
        self.symbol_upper = symbol.upper()
        self.strategies = [] #this stores the strategies used
        self.leverage = 1 #stores current leverage
        self.get_current_invested_amount()
        self.initial_balance = self.get_current_balance() #sotres the initial balance of the session
        self.available_balance = self.initial_balance #stores available balance
        self.last_close_price = 0 #stores last close price
        self.min_units_to_trade = 1e-03
    
    def get_asset(self, symbol):
        if symbol.endswith('busd'): return "BUSD"
        if symbol.endswith('usd'): return "USDT"
        if symbol.endswith('eth'): return "ETH"
        if symbol.endswith('bnb'): return "BNB"
        if symbol.endswith('btc'): return "BTC"
        
    def message_handler(self, msg):
        if 'result' in msg.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        
        QAVol   = float(msg["k"]["q"])
        NoT     = float(msg["k"]["n"])
        TBBAV   = float(msg["k"]["V"])
        TBQAV   = float(msg["k"]["Q"])
        
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        col_num = self.data.shape[1]
        self.data.loc[start_time] = [first, high, low, close, volume,
                                     QAVol, NoT, TBBAV, TBQAV, complete] + [False]*(col_num-10)
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.last_close_price = close
            #print("candle completed", end="")
            self.run_strategy()
        
    def start_streaming(self, interval="1m"):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol=self.symbol.lower() +"_perp",
            id=2,
            interval=interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()
        
    def get_most_recent_data(self, num_candles=100, interval = "1m"):
        #### Get start time for candles ####
        now = datetime.utcnow()
        past = str(now - self.available_intervals(num_candles)[interval])
        #### Request candles and prepare the df ####
        bars = self.client.futures_historical_klines(symbol = self.symbol_upper.lower(), 
                                        interval = interval, 
                                        start_str =past,
                                        end_str = None)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Close Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        use_columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Quote Asset Volume",
                       "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
        df = df[use_columns].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]    
        self.data = df
    
    def start_trading(self, num_candles = 100, interval = "1m", initial_lev = 10,
                     initial_amount = 10, use_prc = True, devs = [2, 4], periods = [60*24],
                     min_std_size = None, min_std_use_prc = False):
        if interval in self.available_intervals(num_candles).keys():
            self.get_most_recent_data(num_candles = num_candles, interval=interval)
            self.prepare_strategies(devs = devs, periods = periods)
            #### STRATEGY PARAMS ####
            self.use_prc = use_prc
            self.initial_amount = initial_amount
            self.curr_amount = self.initial_amount
            self.min_std_size = min_std_size
            self.initial_lev = initial_lev
            self.min_std_use_prc = min_std_use_prc
            self.curr_pos = 0 #strategy positions
            self.prev_pos = 0
            ########################
            self.change_leverage(self.initial_lev)
            self.start_streaming(interval)
        else:
            print("That interval is not available")
            
    def stop_trading(self):
        self.stop_streaming()
        self.go_neutral()
        #print ending metrics here!!
        
    def available_intervals(self, candles_required):
        '''
        Helper function for "get_most_recent_data" method.
        
        '''
        return {
            "1m"  : timedelta(minutes=candles_required),
            "3m"  : timedelta(minutes=candles_required*3),
            "5m"  : timedelta(minutes=candles_required*5),
            "15m" : timedelta(minutes=candles_required*15),
            "30m" : timedelta(minutes=candles_required*30),
            "1h"  : timedelta(hours=candles_required),
            "2h"  : timedelta(hours=candles_required*2),
            "4h"  : timedelta(hours=candles_required*4),
            "6h"  : timedelta(hours=candles_required*6),
            "8h"  : timedelta(hours=candles_required*8),
            "12h" : timedelta(hours=candles_required*12),
            "1d"  : timedelta(days=candles_required),
            "3d"  : timedelta(days=candles_required*3),
            "1w"  : timedelta(days=candles_required*7),
            "1M"  : timedelta(days=candles_required*28) #this may give less than the desired candles because each month has different amount of days
        }
    def prepare_strategies(self, devs, periods):
        if len(periods) == 1:
            periods = periods * len(devs)
        for dev, period in zip(devs,periods):
            self.strategies.append(
                BollingerBands(
                    data = self.data,
                    dev = dev, 
                    periods = period,
                    column = "Close",
                    default_strategy = 1,
                    min_std_size = self.min_std_size,
                    min_std_use_prc_of_sma_mean = self.min_std_use_prc
                )
            )
        for strategy in self.strategies:
            strategy.calculate() #add columns to data 
    
    
    def run_strategy(self):
        self.curr_pos = 0
        for strategy in self.strategies:
            strategy.calculate_for_last_row()
            self.curr_pos += strategy.strategy(-1)
            
        self.predicted_pos = np.sign(self.curr_pos)
        print(" |pp:"+str(self.curr_pos)+str("| "), end="")
        if self.predicted_pos == 0 and np.sign(self.prev_pos) != np.sign(self.curr_pos):
            self.go_neutral()
            self.change_leverage(self.initial_lev)
            self.curr_amount = self.initial_amount
        elif self.predicted_pos == 1 and self.curr_pos != self.prev_pos:
            self.go_neutral(prc = 50)
            self.go_long(amount = self.curr_amount, prc=self.use_prc, go_neutral_first = False)
            #self.change_leverage(self.leverage*1)
            self.curr_amount *= 2
        elif self.predicted_pos == -1 and self.curr_pos != self.prev_pos:
            self.go_neutral(prc = 50)
            self.go_short(amount = self.curr_amount, prc=self.use_prc, go_neutral_first = False)
            #self.change_leverage(self.leverage*1)
            self.curr_amount *= 2
        self.prev_pos = self.curr_pos 
            
    def go_long(self, prc = True, amount = None, go_neutral_first = False):
        if self.get_position() != 0 and go_neutral_first:
            self.go_neutral() #if some position, go neutral first
        if prc: 
            amount = (self.available_balance * amount/100) * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  amount/self.last_close_price
        if quantity < self.min_units_to_trade: quantity = self.min_units_to_trade
        self.create_order(side = "BUY", quantity = quantity)

    def go_short(self, prc = True, amount = None, go_neutral_first = False):
        if self.get_position() != 0 and go_neutral_first:
            self.go_neutral() #if some position, go neutral first
        if prc: 
            amount = (self.available_balance * amount/100) * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  amount/self.last_close_price
        if quantity < self.min_units_to_trade: quantity = self.min_units_to_trade
        self.create_order(side = "SELL", quantity = quantity)
    
    def go_neutral(self, prc = 100):
        #prc between 0 and 100
        prc = min(prc, 100)
        prc = max(0, prc)
        quantity = self.curr_invested_amount*prc/100
        
        is_invested = abs(self.curr_invested_amount) > 0
        quantity_is_less_than_min_to_trade = quantity < self.min_units_to_trade
        if is_invested and quantity_is_less_than_min_to_trade:
            quantity = self.min_units_to_trade
        
        if self.get_position() == 1: #if long, sell all
            self.create_order(side = "SELL", quantity = quantity, reduceOnly = True )
        elif self.get_position() == -1: #if short, buy all
            self.create_order(side = "BUY", quantity = quantity, reduceOnly = True)
    
    def create_order(self, side = "BUY", quantity = 0, reduceOnly = False):
        quantity = round(quantity, 3) #binance accepts max 3 decimals
        if quantity < self.min_units_to_trade: #dont submit invalid orders
            return 
        order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity, reduceOnly = reduceOnly)
        order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
        if order["status"] == "FILLED":
            self.get_current_invested_amount()
            if reduceOnly and self.get_position() == 0:
                print(" |CP| ", end="")
            elif reduceOnly and self.get_position() != 0:
                print(" |CPP| ", end="")    
            else:    
                print(" |OP| ", end="")
        return 0
    
    def get_current_invested_amount(self):
        #IMPORTANT: you wont get notified if a position is liquidated!!!
        #if you want your real time invested amount, call this function before an important action
        #inclues leverage
        infos = self.client.futures_position_information(symbol = self.symbol_upper)
        #use the info of the current pair
        for info in infos:
            if info["symbol"] == self.symbol_upper:
                pos = float(info["positionAmt"])
                self.curr_invested_amount = abs(pos) #invested amount on binance
                self.current_pos = np.sign(pos) #binance pos
                return self.curr_invested_amount
        #not found... no trades then..
        self.curr_invested_amount = 0
        self.current_pos = 0
        return 0
    
    def get_position(self):
        return self.current_pos
    
    def should_end_session(self):
        if self.available_balance < self.initial_balance * 0.3 and self.get_position() == 0: #no money and no positions
            self.stop_trading()
            return True
    def get_current_balance(self):
        balance = pd.DataFrame(self.client.futures_account_balance())# Asset Balance details
        balance = float(balance[ balance["asset"] == self.asset ].iloc[0]["balance"])
        return balance
    
    def change_leverage(self, new_leverage):
        self.client.futures_change_leverage(symbol = self.symbol_upper, leverage = new_leverage)
        self.leverage = new_leverage
        

In [4]:
trader = FuturesTrader(symbol="btcusd", testnet = True)

In [5]:
trader.start_trading(num_candles = 2000, interval = "1m", initial_lev = 20,
                     initial_amount = 10, use_prc = True,
                     devs = [1, 1.5, 2], periods = [60*6],
                    min_std_size = None, min_std_use_prc= True)

.. |pp:0| .. |pp:0| ....... |pp:0| .... |pp:0| ............. |pp:0| ...... |pp:0| .......... |pp:0| .... |pp:0| ..... |pp:0| ........ |pp:0| ... |pp:0| ... |pp:0| ... |pp:0| .... |pp:0| ... |pp:0| .... |pp:0| . |pp:0| .... |pp:0| .............. |pp:0| ....................... |pp:0| .... |pp:0| ..... |pp:0| ....... |pp:0| .. |pp:0| .... |pp:0| ...... |pp:0| .. |pp:0| .... |pp:0| ..... |pp:0| .. |pp:0| ... |pp:0| ..................... |pp:0| ......... |pp:0| ........... |pp:0| ............ |pp:0| ............ |pp:0| .............................. |pp:0| ........ |pp:0| ...................... |pp:0| .. |pp:0| ..... |pp:0| ..... |pp:0| .......... |pp:0| ..... |pp:0| .. |pp:0| ... |pp:0| .... |pp:0| .............................................................. |pp:0| ............................ |pp:0| .................................... |pp:0| .... |pp:0| ........ |pp:0| ... |pp:0| .. |pp:0| .......... |pp:0| .. |pp:0| ..... |pp:0| .................. |pp:0| ............ |pp:0| .. |pp:0| 

.... |pp:-3| ..... |pp:-3| . |pp:-3| .... |pp:-3| ........ |pp:-3| .. |pp:-3| . |pp:-3| .... |pp:-3| ..... |pp:-3| ... |pp:-3| ...... |pp:-3| .......... |pp:-3| ...... |pp:-3| ........... |pp:-3| ............. |pp:-3| ................... |pp:-3| ................... |pp:0|  |CP| .................................... |pp:2|  |OP| ............................................................................... |pp:2| .... |pp:2| ... |pp:2| ........ |pp:3|  |CPP|  |OP| ........ |pp:3| ........................ |pp:3| ........................................................ |pp:3| ......................................................................................................................... |pp:3| ................................................................................... |pp:3| ............ |pp:3| ............ |pp:3| ..... |pp:3| ................... |pp:3| ......................................................... |pp:3| ............. |pp:3| ......... |pp:3| ...... |pp:3| ...

In [6]:
trader.stop_trading()

 |CP| 

In [7]:
trader.data

,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Complete,Close|SMA|360,Close|BBs|1|360|Lower,Close|BBs|1|360|Upper,Close|BBs|1|360|Distance,Close|BBs|1.5|360|Lower,Close|BBs|1.5|360|Upper,Close|BBs|1.5|360|Distance,Close|BBs|2|360|Lower,Close|BBs|2|360|Upper,Close|BBs|2|360|Distance
Date,,,,,,,,,,,,,,,,,,,,
2023-01-06 05:50:00,16804.5,16806.3,16803.5,16803.5,142.516,2.394993e+06,16.0,88.263,1.483335e+06,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06 05:51:00,16805.2,16805.2,16799.4,16799.4,32.972,5.540260e+05,13.0,26.897,4.519668e+05,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06 05:52:00,16801.8,16801.8,16799.4,16799.9,87.353,1.467591e+06,14.0,52.525,8.824985e+05,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06 05:53:00,16799.9,16805.2,16799.9,16803.6,134.943,2.267371e+06,15.0,122.730,2.062180e+06,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06 05:54:00,16803.6,16805.2,16801.9,16805.2,85.732,1.440620e+06,13.0,11.216,1.884871e+05,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-08 01:49:00,16919.4,16922.3,16919.4,16922.3,2131.000,1.259448e+01,75.0,1967.000,1.162523e+01,True,16929.877778,16921.85216,16937.903395,-7.577778,16917.839352,16941.916204,-7.577778,16913.826543,16945.929013,-7.577778
2023-01-08 01:50:00,16922.3,16922.3,16922.3,16922.3,0.000,0.000000e+00,0.0,0.000,0.000000e+00,True,16929.852222,16921.817143,16937.887302,-7.552222,16917.799603,16941.904841,-7.552222,16913.782063,16945.922381,-7.552222
2023-01-08 01:51:00,16922.4,16922.4,16922.4,16922.4,40.000,2.363731e-01,6.0,40.000,2.363731e-01,True,16929.826667,16921.782536,16937.870797,-7.426667,16917.760471,16941.892862,-7.426667,16913.738406,16945.914927,-7.426667


In [ ]:
trader.data.to_csv("data/main.csv")

In [7]:
trader.asset

'USDT'

In [6]:
trader.get_position()

1

..candle completed.........candle completed.......candle completed....candle completed.

### cerca de las 12 horas dio este error:
Unhandled Error
Traceback (most recent call last):
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 96, in callWithLogger
    return callWithContext({"system": lp}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py", line 80, in callWithContext
    return context.call({ILogContext: newCtx}, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 117, in callWithContext
    return self.currentContext().callWithContext(ctx, func, *args, **kw)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py", line 82, in callWithContext
    return func(*args, **kw)
--- <exception caught here> ---
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py", line 683, in _doReadOrWrite
    why = selectable.doRead()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py", line 248, in doRead
    return self._dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py", line 253, in _dataReceived
    rval = self.protocol.dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py", line 330, in dataReceived
    self._flushReceiveBIO()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py", line 296, in _flushReceiveBIO
    ProtocolWrapper.dataReceived(self, bytes)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/policies.py", line 110, in dataReceived
    self.wrappedProtocol.dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 348, in dataReceived
    self._dataReceived(data)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1243, in _dataReceived
    self.consumeData()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1255, in consumeData
    while self.processData() and self.state != WebSocketProtocol.STATE_CLOSED:
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1619, in processData
    fr = self.onFrameEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 1747, in onFrameEnd
    self._onMessageEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 384, in _onMessageEnd
    self.onMessageEnd()
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py", line 647, in onMessageEnd
    self._onMessage(payload, self.message_is_binary)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py", line 387, in _onMessage
    self.onMessage(payload, isBinary)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/websocket/binance_client_protocol.py", line 30, in onMessage
    self.factory.callback(payload_obj)
  File "/tmp/ipykernel_16254/1118553294.py", line 52, in message_handler
    self.run_strategy()
  File "/tmp/ipykernel_16254/1118553294.py", line 182, in run_strategy
    self.go_short(prc = True, amount = 95) # go short with full amount
  File "/tmp/ipykernel_16254/1118553294.py", line 201, in go_short
    self.go_neutral() #if some position, go neutral first
  File "/tmp/ipykernel_16254/1118553294.py", line 214, in go_neutral
    self.create_order(side = "SELL", close_pos = True)
  File "/tmp/ipykernel_16254/1118553294.py", line 222, in create_order
    order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 5985, in futures_create_order
    return self._request_futures_api('post', 'order', True, data=params)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 339, in _request_futures_api
    return self._request(method, uri, signed, True, **kwargs)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 315, in _request
    return self._handle_response(self.response)
  File "/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py", line 324, in _handle_response
    raise BinanceAPIException(response, response.status_code, response.text)
binance.exceptions.BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

WARNING:root:WebSocket connection closed: connection was closed uncleanly ("peer dropped the TCP connection without previous WebSocket closing handshake"), code: 1006, clean: False, reason: connection was closed uncleanly ("peer dropped the TCP connection without previous WebSocket closing handshake")
ERROR:root:Lost connection to Server. Reason: [Failure instance: Traceback: <class 'binance.exceptions.BinanceAPIException'>: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/log.py:80:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py:117:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/python/context.py:82:callWithContext
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py:696:_doReadOrWrite
--- <exception caught here> ---
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/posixbase.py:683:_doReadOrWrite
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py:248:doRead
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/internet/tcp.py:253:_dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py:330:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/tls.py:296:_flushReceiveBIO
/home/mauricio/anaconda3/lib/python3.9/site-packages/twisted/protocols/policies.py:110:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:348:dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1243:_dataReceived
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1255:consumeData
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1619:processData
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:1747:onFrameEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:384:_onMessageEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/websocket/protocol.py:647:onMessageEnd
/home/mauricio/anaconda3/lib/python3.9/site-packages/autobahn/twisted/websocket.py:387:_onMessage
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/websocket/binance_client_protocol.py:30:onMessage
/tmp/ipykernel_16254/1118553294.py:52:message_handler
/tmp/ipykernel_16254/1118553294.py:182:run_strategy
/tmp/ipykernel_16254/1118553294.py:201:go_short
/tmp/ipykernel_16254/1118553294.py:214:go_neutral
/tmp/ipykernel_16254/1118553294.py:222:create_order
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:5985:futures_create_order
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:339:_request_futures_api
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:315:_request
/home/mauricio/anaconda3/lib/python3.9/site-packages/binance/client.py:324:_handle_response
]. Retrying: 1

Old trader:

In [ ]:
class FuturesTrader():
    def __init__(self, symbol="btcusd", testnet = True):
        ####API CONNECTIONS ####
        self.stream = None
        self.client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = testnet)
        #######################
        self.data = None
        self.symbol = symbol
        self.asset = self.get_asset(self.symbol) #get asset like "USDT"
        #### "BTCUSD" is not valid with official api methods, need to use "BTCUSDT" ####
        symbol = self.symbol + "t" if self.symbol.endswith('usd') else self.symbol
        self.symbol_upper = symbol.upper()
        self.strategies = [] #this stores the strategies used
        self.open_orders = []
        self.leverage = 1 #stores current leverage
        self.current_pos = 0 #stores current position
        self.ind_sum = 0 #stores indicators sum
        self.initial_balance = self.get_current_balance() #sotres the initial balance of the session
        self.available_balance = self.initial_balance #stores available balance
        self.last_close_price = 0 #stores last close price
        self.weights_sum = 0 #keeps weight sum of strategies to use it with leverage
        self.rnn = None #stores rnn
        self.lower_bound = 0.003 #Used for rnn strategy and leverage
        self.upper_bound = 0.01
        self.weights = [0.08877842031603478,
  0.05987706928852676,
  0.13120415318448939,
  0.023081971180099897,
  1.2296684936304951,
  0.05058368745150239,
  0.027529078706913866,
  0.017268540027698426,
  0.12069770802559819,
  0.08091687035480966]
    
    def get_asset(self, symbol):
        if symbol.endswith('busd'): return "BUSD"
        if symbol.endswith('usd'): return "USDT"
        if symbol.endswith('eth'): return "ETH"
        if symbol.endswith('bnb'): return "BNB"
        if symbol.endswith('btc'): return "BTC"
        
    def message_handler(self, msg):
        if 'result' in msg.keys(): #skip first message
            return
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        
        QAVol   = float(msg["k"]["q"])
        NoT     = float(msg["k"]["n"])
        TBBAV   = float(msg["k"]["V"])
        TBQAV   = float(msg["k"]["Q"])
        
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        col_num = self.data.shape[1]
        self.data.loc[start_time] = [first, high, low, close, volume,
                                     QAVol, NoT, TBBAV, TBQAV, complete] + [False]*(col_num-10)
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.last_close_price = close
            #print("candle completed", end="")
            self.run_strategy()
        
    def start_streaming(self, interval="1m"):
        self.stream = CMFuturesWebsocketClient()
        self.stream.start()
        self.stream.kline(
            symbol=self.symbol.lower() +"_perp",
            id=2,
            interval=interval,
            callback=self.message_handler,
        )
        
    def stop_streaming(self):
        self.stream.stop()
        
    def get_most_recent_data(self, num_candles=100, interval = "1m"):
        #### Get start time for candles ####
        now = datetime.utcnow()
        past = str(now - self.available_intervals(num_candles)[interval])
        #### Request candles and prepare the df ####
        bars = self.client.futures_historical_klines(symbol = self.symbol_upper.lower(), 
                                        interval = interval, 
                                        start_str =past,
                                        end_str = None)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Close Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        use_columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Quote Asset Volume",
                       "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
        df = df[use_columns].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]    
        self.data = df
    
    def start_trading(self, num_candles = 100, interval = "1m"):
        if interval in self.available_intervals(num_candles).keys():
            self.get_most_recent_data(num_candles = num_candles, interval=interval)
            self.prepare_strategies()
            self.start_streaming(interval)
        else:
            print("That interval is not available")
            
    def stop_trading(self):
        self.stop_streaming()
        self.go_neutral()
        #print ending metrics here!!
        
    def available_intervals(self, candles_required):
        '''
        Helper function for "get_most_recent_data" method.
        
        '''
        return {
            "1m"  : timedelta(minutes=candles_required),
            "3m"  : timedelta(minutes=candles_required*3),
            "5m"  : timedelta(minutes=candles_required*5),
            "15m" : timedelta(minutes=candles_required*15),
            "30m" : timedelta(minutes=candles_required*30),
            "1h"  : timedelta(hours=candles_required),
            "2h"  : timedelta(hours=candles_required*2),
            "4h"  : timedelta(hours=candles_required*4),
            "6h"  : timedelta(hours=candles_required*6),
            "8h"  : timedelta(hours=candles_required*8),
            "12h" : timedelta(hours=candles_required*12),
            "1d"  : timedelta(days=candles_required),
            "3d"  : timedelta(days=candles_required*3),
            "1w"  : timedelta(days=candles_required*7),
            "1M"  : timedelta(days=candles_required*28) #this may give less than the desired candles because each month has different amount of days
        }
    def prepare_strategies(self):
        # prepare params
        #SMA
        SMA_S = 5
        SMA_L = 13
        #EWMA
        approx_avg_period_s = 5
        approx_avg_period_l = 8
        #BBS
        dev = 1.5
        periods = 10
        #prepare strategies
        self.strategies = [
            SMA(
                data = self.data,
                SMA_S = SMA_S,
                SMA_L = SMA_L,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[0]
            ),
            EWMA(
                data = self.data,
                approx_avg_period_s = approx_avg_period_s,
                approx_avg_period_l = approx_avg_period_l,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[1]
            ),
            BollingerBands(
                data = self.data,
                dev = dev, 
                periods = periods,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[2]
            ),
            MACD(
                data = self.data,
                fast =12,
                slow =26, 
                signal = 9,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[3]
            ),
            RSI(
                data = self.data,
                length = 14,
                column = "Close",
                default_strategy = 1,
                weight = self.weights[4]
            ),
            Hammer(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[5]
            ),
            Doji(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[6]
            ),
            EBSW(
                data = self.data,
                default_strategy = 1,
                column = "Close",
                weight = self.weights[7]
            ),
            ADX(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[8],
            ),
            KVO(
                data = self.data,
                default_strategy = 1,
                weight = self.weights[9],
            ),
            OBV(
                data = self.data
            )
        ]
        for strategy in self.strategies:
            strategy.calculate() #add columns to data 
        #self.data.dropna(inplace = True) #dropna after calculating the strategy
        self.rnn = RNN(
            data = self.data,
            default_strategy = 1,
            weight = 1,
            model = 'models/rnn_15.h5',
            scaler = 'models/scaler_dic19_1m.pkl'
        )
        self.rnn.calculate() 
        #self.calculate_weights_sum()
    
    def leverage_strategy(self, min_x, max_x, min_y = 1, max_y = 2):
        if max_y < min_y:
            print("max leverage is less than min leverage")
            return -1
        #y = y1 + m(x-x1) , m =(y2-y1)/(x2-x1)
        #two points of the line: (min_x, min_y) and (max_x, max_y)
        x = min(max_x, abs(self.ind_sum)) #short or long, we want just abs number
        m = (max_y-min_y)/(max_x-min_x)
        new_leverage = round( min_y + m*( x - min_x ))
        self.client.futures_change_leverage(symbol = self.symbol_upper, leverage = new_leverage)
        self.leverage = new_leverage
        return new_leverage
    
    
    def run_strategy(self):
        self.ind_sum = 0
        for strategy in self.strategies:
            strategy.calculate_for_last_row()
        #    self.ind_sum += strategy.strategy(-1) * strategy.weight
        #if abs(self.ind_sum) < 0.5: self.ind_sum = 0 #discards weak signals < |0.5|    
        #self.predicted_pos = np.sign(self.ind_sum)
        self.rnn.calculate_for_last_row()
        self.predicted_pos, self.ind_sum = self.rnn.strategy(-1, bound = self.bound) 
        print(" |pp:"+str(self.predicted_pos)+str("| "), end="")
        if self.predicted_pos == 0:
            self.go_neutral()
        elif self.predicted_pos == 1 and self.get_position() in [0, -1]:
            self.go_long(prc = True, amount = 95) # go long with full amount
        elif self.predicted_pos == -1 and self.get_position() in [0, 1]:
            self.go_short(prc = True, amount = 95) # go short with full amount
            
    def go_long(self, prc = True, amount = None):
        if self.get_position() != 0:
            self.go_neutral() #if some position, go neutral first
        if self.should_end_session(): return
        self.leverage_strategy(min_x = self.lower_bound, max_x = self.upper_bound)
        if prc == True: 
            amount = (self.available_balance * amount/100) * self.leverage
        else:
            amount = amount * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  round(amount/self.last_close_price, 3)
        self.create_order(side = "BUY", quantity = quantity)

    def go_short(self, prc = True, amount = None):
        if self.get_position() != 0:
            self.go_neutral() #if some position, go neutral first
        if self.should_end_session(): return
        self.leverage_strategy(min_x = self.lower_bound, max_x = self.upper_bound)
        if prc == True: 
            amount = (self.available_balance * amount/100) * self.leverage
        else:
            amount = amount * self.leverage
        #amount = self.validate_order_amount(amount, self.leverage)
        quantity =  round(amount/self.last_close_price, 3)
        self.create_order(side = "SELL", quantity = quantity)
    
    def go_neutral(self):
        if self.get_position() == 1: #if long, sell all
            self.create_order(side = "SELL", close_pos = True)
        elif self.get_position() == -1: #if short, buy all
            self.create_order(side = "BUY", close_pos = True)
    
    def create_order(self, side = "BUY", quantity = 0, close_pos = False):
        if close_pos:
            open_order = self.open_orders[0]
            quantity = open_order["executedQty"]
            order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity)
            order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
            self.open_orders = []
            self.current_pos = 0
            self.available_balance = self.get_current_balance()
            print(" |CP| ", end="")
            return 0
        order_open = self.client.futures_create_order(symbol = self.symbol_upper, side = side,
                                         type = "MARKET", quantity = quantity)
        order = self.client.futures_get_order(symbol = self.symbol_upper, orderId = order_open["orderId"])
        self.open_orders.append(order)
        self.current_pos = 1 if side == "BUY" else -1
        print(" |OP| ", end="")
        return 0
    
    def get_position(self):
        return self.current_pos
    
    def should_end_session(self):
        if self.available_balance < self.initial_balance * 0.3 and self.get_position() == 0: #no money and no positions
            self.stop_trading()
            return True
    def get_current_balance(self):
        balance = pd.DataFrame(self.client.futures_account_balance())# Asset Balance details
        balance = float(balance[ balance["asset"] == self.asset ].iloc[0]["balance"])
        return balance
    
    def calculate_weights_sum(self):
        self.weights_sum = 0
        for strategy in self.strategies:
            self.weights_sum += abs(strategy.weight)
            
            